In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/demres

/Users/zurfarosa/demres


In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.common import codelists,druglists
from demres.common.process_pt_features import *
from demres.common.process_entries import *
from demres.demins.constants import Study_Design as sd
from demres.demins.statistical_functions import *
from common.helper_functions import *

In [4]:
pd.set_option('display.max_columns', None)

## Process raw CSV files

In [5]:
# create_pegmed()

In [6]:
# create_pegprod()

In [7]:
# create_prescriptions()

In [8]:
# create_consultations()

In [9]:
# create_clinicals()

In [10]:
# create_tests()

In [11]:
# create_referrals()

In [12]:
# create_immunisations()

In [13]:
# create_medcoded_entries()

## Create basic pt_features dataframe
*pt_features will contain all the variables (e.g. age, male gender, insomnia count) used in the logistic regression*

In [5]:
# specify subtype of dementia we're interested in - 'vascular','alzheimers' or 'all_dementia'
subtype = 'all_dementia' 

In [24]:
all_entries = get_all_entries()

In [ ]:
pt_features = create_pt_features()

In [17]:
pt_features = get_index_date_and_caseness_and_add_final_dementia_subtype(all_entries,pt_features)

In [18]:
pt_features = only_include_specific_dementia_subtype(pt_features,subtype=subtype)

In [19]:
pt_features = add_data_start_and_end_dates(all_entries,pt_features)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [20]:
len(pt_features[pt_features['isCase']==True]),len(pt_features[pt_features['isCase']==False])

(47480, 47020)

In [21]:
pt_features.head(3)

,patid,yob,pracid,male,index_date,isCase,final dementia medcode,data_start,data_end
0,57001,32,001,0,2006-12-05,True,19477.0,1997-03-07,2013-05-17
1,60001,24,001,0,NaT,False,NaN,1997-03-07,2013-05-20
2,149001,19,001,1,2004-07-02,True,7572.0,1997-03-07,2004-11-17


In [22]:
pt_features.to_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'.csv',index=False)

In [23]:
for window in sd.exposure_windows:
    print(window['name'],' being matched')    
    pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)
    pt_features = match_cases_and_controls(pt_features,req_yrs_post_index=sd.req_yrs_post_index,start_year=abs(window['start_year']))
    pt_features = delete_unmatched_cases_and_controls(pt_features)
    pt_features.to_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ window['name'] +'.csv',index=False)

12_to_7  being matched


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/zurfarosa/demres/common/process_pt_features.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

10_to_5  being matched
8_to_3  being matched


## Add derived variables to pt_features 
*e.g. insomnia count, presence of diabetes, consultation count*

In [12]:
medcoded_entries = pd.read_hdf('hdf/medcoded_entries.hdf')

In [11]:
prescriptions = pd.read_hdf('hdf/prescriptions.hdf')

In [26]:
# Add condition status (e.g. insomnia count, presence of diabetes, presence of stroke)
for window in sd.exposure_windows:
    print(window['name'],'...')
    pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ window['name'] +'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)
#     pt_features = get_multiple_condition_statuses(pt_features,medcoded_entries,window,codelists.codelist_list_of_lists)
#     pt_features = create_pdds(pt_features,prescriptions,window,druglists.psychotropic_list_of_lists)
    pt_features = get_consultation_count(pt_features,all_entries,window)
    pt_features.to_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ window['name'] +'.csv',index=False)

12_to_7 ...
10_to_5 ...
8_to_3 ...


In [23]:
for window in sd.exposure_windows:
    pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ window['name'] +'.csv',delimiter=',')
    pt_features.drop(['non_insomnia_GP_consultations'],axis=1,inplace=True)
    pt_features.to_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ window['name'] +'.csv',index=False)